In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage.filters import gaussian
from skimage.segmentation import active_contour
import math
import imutils
import pandas as pd
import csv
from pathlib import Path
from tqdm.auto import trange, tqdm
from scipy import interpolate
from scipy.optimize import fsolve
# import enlighten

## <span style="color:orange">*Image Preprocessing*</span>

### <span style="color:#AF7AC5">Import Image</span>

In [ ]:
def read_image(path):
    
    img = cv2.imread(path)
    
    gray_eye_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray_eye_image

### <span style="color:#AF7AC5">Iris Localization</span>

#### Houghman Circle Detection

##### Find Pupil `Slow`

In [ ]:
def get_edges(image):
        edges = cv2.Canny(image, 20, 100)
        kernel = np.ones((3, 3), np.uint8)
        edges = cv2.dilate(edges, kernel, iterations=2)
        # ksize = 2 * random.randrange(5, 11) + 1
        edges = cv2.GaussianBlur(edges, (15, 15), 0)
        return edges

In [ ]:
a = np.ones((3, 3), np.uint8)
print(a)

In [ ]:
def find_pupil(img):
    param1 = 200  # 200
    param2 = 120  # 150
    pupil_circles = []
    while(param2 > 35 and len(pupil_circles) < 100):
        for mdn, thrs in [(m, t)
                          for m in [3, 5, 7]
                          for t in [20, 25, 30, 35, 40, 45, 50, 55, 60]]:
            # Median Blur
            median = cv2.medianBlur(img, 2*mdn+1)

            # Threshold
            _, thres = cv2.threshold(
                median, thrs, 255,
                cv2.THRESH_BINARY_INV)

            # Canny Edges
            edges = get_edges(thres)

            # HoughCircles
            circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 1,
                                       np.array([]), param1, param2)
            if circles is not None:
                # convert the (x, y) coordinates and radius of the circles to integers
                circles = np.round(circles).astype("int")
                for c in circles:
                    pupil_circles.append(c)

        param2 = param2 - 10

    cimg = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
    if circles is None:
        return
    mean_0 = int(np.mean([c[0][0] for c in circles]))
    mean_1 = int(np.mean([c[0][1] for c in circles]))
    mean_2 = int(np.mean([c[0][2] for c in circles]))

    return mean_0, mean_1, mean_2


##### Find Pupil `Fast`

In [ ]:
def find_pupil_new(img):
    img = cv2.medianBlur(img, 15)
    img = cv2.Canny(img, 0, 50)
    param1 = 200  # 200
    param2 = 120  # 150
    decrement = 1
    circles = None
    while circles is None and param2 > 20:
        # HoughCircles
        circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1, 1,
                                    param1=param1, param2=param2,
                                    minRadius=20, maxRadius=80)

        if circles is not None:
            break

        param2 -= decrement
    
    if circles is None:
        return None, None, None
    
    return circles.astype(int)[0][0]


#### Localization

##### Houghman circle with Active contour

|`alpha`|`beta`|`gamma`|
|---|---|---|
|1.6|390|0.005|
|1.5|390|0.005|
|1.6|300|0.05|
|1.6|500|0.05|
|1.6|1000|0.05|
|1.45|590|0.01|

In [ ]:
def localization(img, N=400, alpha=1.45, beta=590, w_line=0.01, gamma=0.01):
    DoG = cv2.GaussianBlur(img, (3, 3), 0) - cv2.GaussianBlur(img, (25, 25), 0)
    median1 = cv2.medianBlur(DoG, 9)
    eroted = cv2.erode(median1, np.ones((3, 3), np.uint8), iterations=1)
    median2 = cv2.medianBlur(eroted, 5)
    dilated = cv2.dilate(median2, np.ones((3, 3), np.uint8), iterations=1)
    eroted = cv2.erode(dilated, np.ones((5, 5), np.uint8), iterations=1)
    result = cv2.bitwise_or(img, eroted)
        
    x, y, rad = find_pupil_new(img)
    if x is None:
        x, y = 350, 250
    
    s = np.linspace(0, 2*np.pi, 400)
    c = x + 150*np.cos(s)
    r = y + 150*np.sin(s)
    init = np.array([r, c]).T

    snake = active_contour(result, init, alpha=alpha, beta=beta, w_line=w_line, gamma=gamma)
    
    return init, snake, (x, y, rad)
    

##### Only Active contour

In [ ]:
def localization_active(img, N=400):
    DoG = cv2.GaussianBlur(img, (3, 3), 0) - cv2.GaussianBlur(img, (25, 25), 0)
    median1 = cv2.medianBlur(DoG, 9)
    eroted = cv2.erode(median1, np.ones((3, 3), np.uint8), iterations=1)
    median2 = cv2.medianBlur(eroted, 5)
    dilated = cv2.dilate(median2, np.ones((3, 3), np.uint8), iterations=1)
    eroted = cv2.erode(dilated, np.ones((5, 5), np.uint8), iterations=1)
    result = cv2.bitwise_or(img, eroted)
        
    s = np.linspace(0, 2*np.pi, 400)
    c = 350 + 200*np.cos(s)
    r = 250 + 200*np.sin(s)

    init = np.array([r, c]).T
    
    temp = cv2.medianBlur(img, 15)
    temp = cv2.Canny(temp, 0, 20)

    snake_pupil = active_contour(result, init, alpha=4, beta=1000, gamma=0.05)
    
    s = np.linspace(0, 2*np.pi, 400)
    c = 350 + 250*np.cos(s)
    r = 250 + 250*np.sin(s)
    init = np.array([r, c]).T

    snake = active_contour(result, init, alpha=1.45, beta=590, gamma=0.05)
    
    return init, snake, snake_pupil

##### AI Generated Active Contours

In [ ]:
def localize_iris(iris_img):
  # Convert the image to grayscale
  gray_iris = cv2.cvtColor(iris_img, cv2.COLOR_BGR2GRAY)

  # Apply a Gaussian blur to the image to reduce noise
  blurred_iris = cv2.GaussianBlur(gray_iris, (5, 5), 0)

  # Use the Canny edge detector to find edges in the image
  edges = cv2.Canny(blurred_iris, 50, 150)

  # Use the active contour algorithm (e.g. Snake) to find the iris boundary
  contours = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  contours = imutils.grab_contours(contours)
  contours = sorted(contours, key = cv2.contourArea, reverse = True)[:1]
  iris_boundary = cv2.approxPolyDP(contours[0], 0.01 * cv2.arcLength(contours[0], True), True)

  # Use the iris boundary to crop the iris region from the image
  mask = np.zeros(edges.shape, dtype=np.uint8)
  cv2.drawContours(mask, [iris_boundary], -1, 255, -1)
  cropped_iris = cv2.bitwise_and(iris_img, iris_img, mask=mask)

  # Return the resulting cropped iris image
  return cropped_iris

### <span style="color:#AF7AC5">Iris Normalization</span>

#### Circle Function

In [ ]:
def trans_axis(circle, theta):

    x0, y0, r = circle
    x = int(x0 + r * math.cos(theta))
    y = int(y0 + r * math.sin(theta))
    return x, y

#### Houghman Circle with Active Contour

In [ ]:
def normalization(img, pupil_circle, iris_circle, M=64, N=400, offset=0):

    normalized = np.zeros((M, N))
    theta = np.linspace(0, 2 * np.pi, N)

    for i in range(N):
        curr_theta = theta[i] + offset
        if curr_theta > 2 * np.pi:
            curr_theta -= 2 * np.pi
        begin = trans_axis(pupil_circle, curr_theta)
        end = iris_circle

        xspace = np.linspace(begin[0], end[i][0], M)
        yspace = np.linspace(begin[1], end[i][1], M)
        normalized[:, i] = [255 - img[int(y), int(x)]
                            if 0 <= int(x) < img.shape[1] and 0 <= int(y) < img.shape[0]
                            else 0
                            for x, y in zip(xspace, yspace)]
    return normalized

#### Active Contour Only

In [ ]:
def normalization_active(img, pupil_circle, iris_circle, M=64, N=400, offset=0):

    normalized = np.zeros((M, N))
    theta = np.linspace(0, 2 * np.pi, N)

    for i in range(N):
        curr_theta = theta[i] + offset
        if curr_theta > 2 * np.pi:
            curr_theta -= 2 * np.pi
        begin = pupil_circle
        end = iris_circle

        xspace = np.linspace(begin[i][0], end[i][0], M)
        yspace = np.linspace(begin[i][1], end[i][1], M)
        normalized[:, i] = [255 - img[int(y), int(x)]
                            if 0 <= int(x) < img.shape[1] and 0 <= int(y) < img.shape[0]
                            else 0
                            for x, y in zip(xspace, yspace)]
    return normalized

### <span style="color:#AF7AC5">Eye Lash Removal</span>

In [ ]:
def lash_removal(img, thresh=40):
    ref = img < thresh
    coords = np.where(ref == 1)
    rmov_img = img.astype(float)
    rmov_img[coords] = float('nan')
    return rmov_img

In [ ]:
def lash_removal_daugman(img, thresh=40):
    ref = img < thresh
    coords = np.where(ref == 1)
    rmov_img = img.astype(float)
    rmov_img[coords] = float('nan')
    temp_img = rmov_img.copy()
    temp_img[coords] = 255/2
    avg = np.sum(temp_img) / (rmov_img.shape[0] * rmov_img.shape[1])
    rmov_img[coords] = avg
    
    noise_img = np.zeros(img.shape)
    noise_img[coords] = 1
    return rmov_img, noise_img.astype(bool)

## <span style="color:orange">*Feature Extraction*</span>

### <span style="color:#AF7AC5">Convolute with Gabor</span>

In [ ]:
def gaborconvolve_f(img, minw_length, mult, sigma_f):
    """
    Convolve each row of an imgage with 1D log-Gabor filters.
    """
    rows, ndata = img.shape
    logGabor_f = np.zeros(ndata)
    filterb = np.zeros([rows, ndata], dtype=complex)

    radius = np.arange(ndata/2 + 1) / (ndata/2) / 2
    radius[0] = 1

    # filter wavelength
    wavelength = minw_length

    # radial filter component 
    fo = 1 / wavelength
    logGabor_f[0: int(ndata/2) + 1] = np.exp((-(np.log(radius/fo))**2) /
                                    (2 * np.log(sigma_f)**2))
    logGabor_f[0] = 0

    # convolution for each row 
    # Not optimized version
    # for r in range(rows):
    #     signal = img[r, 0:ndata]
    #     imagefft = np.fft.fft(signal)
    #     filterb[r, :] = np.fft.ifft(imagefft * logGabor_f)

    # Optimized version
    signals = img[:, 0:ndata]
    imagefft = np.fft.fft(signals, axis=1)
    filterb = np.fft.ifft(imagefft * logGabor_f, axis=1)
    
    return filterb

In [ ]:
def encode_iris(arr_polar, arr_noise, minw_length, mult, sigma_f):
    """
    Generate iris template and noise mask from the normalised iris region.
    """
    # convolve with gabor filters
    filterb = gaborconvolve_f(arr_polar, minw_length, mult, sigma_f)
    l = arr_polar.shape[1]
    template = np.zeros([arr_polar.shape[0], 2 * l])
    h = np.arange(arr_polar.shape[0])

    # making the iris template
    mask_noise = np.zeros(template.shape)
    filt = filterb[:, :]

    # quantization and check to se if the phase data is useful
    H1 = np.real(filt) > 0
    H2 = np.imag(filt) > 0

    H3 = np.abs(filt) < 0.0001
    # Not optimized code
    for i in range(l):
        ja = 2 * i

        # biometric template
        template[:, ja] = H1[:, i]
        template[:, ja + 1] = H2[:, i]
        # noise mask_noise
        mask_noise[:, ja] = arr_noise[:, i] | H3[:, i]
        mask_noise[:, ja + 1] = arr_noise[:, i] | H3[:, i]


    return template, mask_noise

## <span style="color:orange">*Matching*</span>

### <span style="color:#AF7AC5">Shift Bit Function</span>

In [ ]:
def shiftbits_ham(template, noshifts):
    templatenew = np.zeros(template.shape)
    width = template.shape[1]
    s = 2 * np.abs(noshifts)
    p = width - s

    if noshifts == 0:
        templatenew = template

    elif noshifts < 0:
        x = np.arange(p)
        templatenew[:, x] = template[:, s + x]
        x = np.arange(p, width)
        templatenew[:, x] = template[:, x - p]

    else:
        x = np.arange(s, width)
        templatenew[:, x] = template[:, x - s]
        x = np.arange(s)
        templatenew[:, x] = template[:, p + x]

    return templatenew

### <span style="color:#AF7AC5">Hamming Distance</span>

In [ ]:
# def HammingDistance(template1, mask1, template2, mask2):
#     hd = np.nan

#     # Shifting template left and right, use the lowest Hamming distance
#     for shifts in range(-8, 9):
#         template1s = shiftbits_ham(template1, shifts)
#         mask1s = shiftbits_ham(mask1, shifts)

#         mask = np.logical_and(mask1s, mask2)
#         nummaskbits = np.sum(mask == 1)
#         totalbits = template1s.size - nummaskbits

#         C = np.logical_xor(template1s, template2)
#         C = np.logical_and(C, np.logical_not(mask))
#         bitsdiff = np.sum(C == 1)

#         if totalbits == 0:
#             hd = np.nan
#         else:
#             hd1 = bitsdiff / totalbits
#             if hd1 < hd or np.isnan(hd):
#                 hd = hd1

#     return hd

In [ ]:
def HammingDistance(template1, mask1, template2, mask2):
    hd = np.nan

    bitsdiff_arr = np.empty(17, dtype=np.float64)
    totalbits_arr = np.empty(17, dtype=np.float64)

    for i, shifts in enumerate(range(-8, 9)):
        template1s = shiftbits_ham(template1, shifts)
        mask1s = shiftbits_ham(mask1, shifts)

        mask = np.logical_and(mask1s, mask2)
        nummaskbits = np.sum(mask == 1)
        totalbits_arr[i] = template1s.size - nummaskbits

        C = np.logical_xor(template1s, template2)
        C = np.logical_and(C, np.logical_not(mask))
        bitsdiff_arr[i] = np.sum(C == 1)

    for i, totalbits in enumerate(totalbits_arr):
        if totalbits == 0:
            hd = np.nan
        else:
            hd1 = bitsdiff_arr[i] / totalbits
            if hd1 < hd or np.isnan(hd):
                hd = hd1

    return hd


## <span style="color:orange">*Test Plot*</span>

### <span style="color:#AF7AC5">All Image Check</span>

In [ ]:
def plot_prepro(img_num1=0, img_num2=60, plot_cols=3):
    plot_size = 29
    plot_rows = math.ceil((img_num2-img_num1) / plot_cols)

    # fig, ax = plt.subplots(plot_rows*2, plot_cols, figsize=(20, plot_size*plot_rows))
    fig = plt.figure(figsize=(20, plot_size*plot_rows), constrained_layout=False)
    outer_grid = fig.add_gridspec(plot_rows*2, plot_cols, wspace=0.1, hspace=-0.2)

    for i in range(img_num1, img_num2):
        # Read image
        img = read_image(f'Iris-Dataset/CASIA-IrisV2/device1/00{str(i).zfill(2)}/00{str(i).zfill(2)}_000.bmp')
        
        # Image Preprocessing (Localization)
        init, snake, circles = localization(img)
        
        pupil_circle = circles
        iris_circle = np.flip(np.array(snake).astype(int), 1)
        
        # Graph of Localization
        x = i//plot_cols
        y = i-(i//plot_cols)*plot_cols
        
        inner_grid = outer_grid[x, y].subgridspec(5, 1, hspace=-0.75)
        ax0 = fig.add_subplot(inner_grid[0:2])
        ax1 = fig.add_subplot(inner_grid[3])
        ax2 = fig.add_subplot(inner_grid[4])
        
        ax0.imshow(img, cmap='gray')
        ax0.plot(snake[:, 1], snake[:, 0], '-b', lw=1)
        ax0.set_title(f'Image {i}')
        
        if circles[2] is None:
            print(f"No circles found in image {i}")
            ax1.imshow(img, cmap='gray')
            ax1.axis([0, 400, 64, 0])
            continue
        
        circle = plt.Circle((circles[0], circles[1]), circles[2], color='g', fill=False, linewidth=1)
        ax0.add_patch(circle)
        ax0.scatter(circles[0], circles[1], s=20, c='g', marker='o')
        
        # Image Preprocessing (Normalization)
        # With eyelashes removal
        iris_norm = normalization(img, pupil_circle, iris_circle)
        
        # Without eyelashes removal
        # iris_norm = normalization(img, pupil_circle, iris_circle)
        
        ax1.imshow(iris_norm, cmap='gray')
        ax1.set_title(f'Normalized Image {i}')
        
        # Feature Extraction
        romv_img, noise_img = lash_removal_daugman(iris_norm, thresh=50)
        template, mask_noise = encode_iris(romv_img, noise_img, minw_length=18, mult=1, sigma_f=0.5)
        
        ax2.imshow(template, cmap='gray')
        ax2.set_title(f'Binary Encoded Image {i}')
        
    plt.show()

### <span style="color:#AF7AC5">Matching Check</span>

In [ ]:
def plot_matching(img_num=0, compare_num=10):
    plot_size = 29
    plot_cols = 2
    plot_rows = math.ceil((compare_num) / plot_cols)
    templates = []
    masks = []

    # fig, ax = plt.subplots(plot_rows*2, plot_cols, figsize=(20, plot_size*plot_rows))
    fig = plt.figure(figsize=(20, plot_size*plot_rows), constrained_layout=False)
    outer_grid = fig.add_gridspec(plot_rows*2, plot_cols, wspace=0.1, hspace=-0.2)

    for i in range(compare_num):
        # Read image
        img = read_image(f'Iris-Dataset/CASIA-IrisV2/device1/00{str(img_num).zfill(2)}/00{str(img_num).zfill(2)}_0{str(i).zfill(2)}.bmp')
        
        # Image Preprocessing (Localization)
        init, snake, circles = localization(img)
        
        pupil_circle = circles
        iris_circle = np.flip(np.array(snake).astype(int), 1)
        
        # Graph of Localization
        x = i//plot_cols
        y = i-(i//plot_cols)*plot_cols
        
        inner_grid = outer_grid[x, y].subgridspec(5, 1, hspace=-0.7)
        ax0 = fig.add_subplot(inner_grid[0:2])
        ax1 = fig.add_subplot(inner_grid[3])
        ax2 = fig.add_subplot(inner_grid[4])
        
        ax0.imshow(img, cmap='gray')
        ax0.plot(snake[:, 1], snake[:, 0], '-b', lw=1)
        ax0.set_title(f'Image {img_num}', fontsize=20)
        
        if circles[2] is None:
            print(f"No circles found in image {img_num}")
            ax1.imshow(img, cmap='gray')
            ax1.axis([0, 400, 64, 0])
            continue
        
        circle = plt.Circle((circles[0], circles[1]), circles[2], color='g', fill=False, linewidth=1)
        ax0.add_patch(circle)
        ax0.scatter(circles[0], circles[1], s=20, c='g', marker='o')
        
        # Image Preprocessing (Normalization)
        # With eyelashes removal
        iris_norm = normalization(img, pupil_circle, iris_circle)
        
        # Without eyelashes removal
        # iris_norm = normalization(img, pupil_circle, iris_circle)
        
        ax1.imshow(iris_norm, cmap='gray')
        ax1.set_title(f'Normalized Image', fontsize=20)
        
        # Feature Extraction
        romv_img, noise_img = lash_removal_daugman(iris_norm, thresh=50)
        template, mask_noise = encode_iris(romv_img, noise_img, minw_length=18, mult=1, sigma_f=0.5)
        
        templates.append(template)
        masks.append(mask_noise)
        
        ax2.imshow(template, cmap='gray')
        ax2.set_title(f'Binary Encoded Image', fontsize=20)
        
        # Matching
        if len(templates) >= 2:
            hd_raw = HammingDistance(templates[0], masks[0], templates[i], masks[i])
            result = 'Match' if hd_raw <= 0.49 else 'Not Match'
            ax0.set_title(f'Hamming Distance: {hd_raw} ➜ {result}', fontsize=20)
        
    # print(f'Hamming Distance: {hd_raw}')
    # print(f'Matching Result: {hd_raw >= 0.5}')
        
    plt.show()

### <span style="color:#AF7AC5">Accuracy Plot</span>

In [ ]:
def acc_plot(img_num=1, compare_num=20, ref_img='Iris-Dataset/CASIA-IrisV2/device1/0000/0000_000.bmp'):
    templates = []
    masks = []
    df = pd.DataFrame(columns=[], index=[f"Image {i}" for i in range(compare_num)])
    
    # Read image
    ref_img = read_image(ref_img)
    
    # Image Preprocessing (Localization)
    init, snake, circles = localization(ref_img)
    
    pupil_circle = circles
    iris_circle = np.flip(np.array(snake).astype(int), 1)
    
    if circles[2] is None:
        print(f"No circles found in image {img_num}")
        return
    
    # Image Preprocessing (Normalization)
    iris_norm = normalization(ref_img, pupil_circle, iris_circle)
    
    # Feature Extraction
    romv_img, noise_img = lash_removal_daugman(iris_norm, thresh=50)
    template, mask_noise = encode_iris(romv_img, noise_img, minw_length=18, mult=1, sigma_f=0.5)
    
    templates.append(template)
    masks.append(mask_noise)

    for i in range(img_num):
        hd_raw = []
        for j in range(compare_num):
            # Read image
            img = read_image(f'Iris-Dataset/CASIA-IrisV2/device1/00{str(i).zfill(2)}/00{str(i).zfill(2)}_0{str(j).zfill(2)}.bmp')
            
            # Image Preprocessing (Localization)
            init, snake, circles = localization(img)
            
            pupil_circle = circles
            iris_circle = np.flip(np.array(snake).astype(int), 1)
            
            if circles[2] is None:
                print(f"No circles found in image {img_num}")
                continue
            
            # Image Preprocessing (Normalization)
            iris_norm = normalization(img, pupil_circle, iris_circle)
            
            # Feature Extraction
            romv_img, noise_img = lash_removal_daugman(iris_norm, thresh=50)
            template, mask_noise = encode_iris(romv_img, noise_img, minw_length=18, mult=1, sigma_f=0.5)
            
            templates.append(template)
            masks.append(mask_noise)
            
            # Matching
            hd_raw.append(HammingDistance(templates[0], masks[0], templates[i*20+j+1], masks[i*20+j+1]))

        df[f'Folder {i}'] = hd_raw
        
    return df

### <span style="color:#AF7AC5">Accuracy Calculation</span>

In [ ]:
def write_iris_data_to_csv(col, rows, archive,test_fol=100,test_img=10):
  with open(archive, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(col)
        writer.writerows(rows)

In [ ]:
def calc_classification_metrics(truth, prediction):

  if prediction == 1:
    if truth == 1:
      return "TP"
    else:
      return "FP"
  elif truth == 1:
    return "FN"
  else:
    return "TN"

In [ ]:
test_img = 10 
rows = [['Image' + str(i)] for i in range(test_img)]
print(rows)

In [ ]:
def acc_calc_demo(ref_imgs=0, ref_img=0, test_fol=100, test_img=10):  
    col = ['FolderName'] + ['folder' + str(i) + 'L' if j % 2 == 0 else 'folder' + str(i) + 'R' for i in range(test_fol) for j in range(2) ]
    rows = [['Image' + str(i)] for i in range(test_img)]
    
    img_1_L = read_image(
        f'Iris-Dataset/CASIA-Iris-Thousand/{str(ref_imgs).zfill(3)}/L/S5{str(ref_imgs).zfill(3)}L{str(ref_img).zfill(2)}.jpg')

    img_1_R = read_image(
        f'Iris-Dataset/CASIA-Iris-Thousand/{str(ref_imgs).zfill(3)}/R/S5{str(ref_imgs).zfill(3)}R{str(ref_img).zfill(2)}.jpg')

    error = 0
    
    folders = tqdm(range(test_fol), unit='folder', position=0, dynamic_ncols=True,)
    images = tqdm(range(test_img), unit='image', position=1, leave=False, dynamic_ncols=True)
    
    for folder in range(test_fol):
        folders.set_description(f"Processing Folder {folder}")
        folders.update()
        
        for num_img in range(test_img):
            images.set_description(f"Processing Image {num_img}")
            images.update()
            
            if folder == ref_imgs and num_img == ref_img:
                continue
            img_2_L = read_image(
                f'Iris-Dataset/CASIA-Iris-Thousand/{str(folder).zfill(3)}/L/S5{str(folder).zfill(3)}L{str(num_img).zfill(2)}.jpg')

            img_2_R = read_image(
                f'Iris-Dataset/CASIA-Iris-Thousand/{str(folder).zfill(3)}/R/S5{str(folder).zfill(3)}R{str(num_img).zfill(2)}.jpg')

            imgs = [[img_1_L, img_1_R], [img_2_L, img_2_R]]
            templates = [[], []]
            masks = [[], []]
            results = []

            for i in range(len(imgs)):
                for j in range(len(imgs[i])):
                    img = imgs[i][j]

                    _, snake, circles = localization(img, N=400)

                    pupil_circle = circles
                    iris_circle = np.flip(np.array(snake).astype(int), 1)

                    if circles[2] is None:
                        error += 1
                        break

                    # Image Preprocessing (Normalization)
                    iris_norm = normalization(img, pupil_circle, iris_circle)

                    # Feature Extraction
                    romv_img, noise_img = lash_removal_daugman(
                        iris_norm, thresh=50)
                    template, mask_noise = encode_iris(
                        romv_img, noise_img, minw_length=18, mult=1, sigma_f=0.5)
                    


                    templates[i].append(template)
                    masks[i].append(mask_noise)

                    # Matching
                    if len(templates[1]) > 0:
                        hd_raw = HammingDistance(
                            templates[i-1][j], masks[i-1][j], templates[i][j], masks[i][j])
                        results.append(hd_raw)

            if len(results) == 2:
                for value in results:
                    rows[num_img].append(value)
                
                if results[0] <= 0.48 and results[1] <= 0.48:
                    if folder == ref_imgs:
                        continue
                    else:
                        error += 1
                elif (results[0] <= 0.49 and results[1] <= 0.45) or (results[1] <= 0.49 and results[0] <= 0.45):
                    if folder == ref_imgs:
                        continue
                    else:
                        error += 1
                else:
                    continue
            else:
                error += 1
                print(f'Folder {folder} Image {num_img}: {results}')
                for j in range(1):
                    rows[num_img].append(np.nan)
    
        images.reset()

    write_iris_data_to_csv(col, rows,'output2.csv', test_fol, test_img)
        
           
    return error/(test_fol * test_img-1)*100


In [ ]:
def acc_calc_demo2(ref_imgs=0, ref_img=0, test_fol=100, test_img=10, filename = 'test.csv'):    

    col = ['FolderName'] + ['folder' + str(i) for i in range (0,1000)  ]
    rows = [['Image' + str(i)] for i in range(test_img)]
    data1 = []
    
    img_1 = read_image(
        f'archive/{str(ref_imgs).zfill(3)}/S6{str(ref_imgs).zfill(3)}S{str(ref_img).zfill(2)}.jpg')
    


    error = 0
    
    folders = tqdm(range(test_fol), unit='folder', position=0, dynamic_ncols=True,)
    images = tqdm(range(test_img), unit='image', position=1, leave=False, dynamic_ncols=True)
    
    for folder in range(test_fol):
        folders.set_description(f"Processing Folder {folder}")
        folders.update()
        
        for num_img in range(test_img):
            images.set_description(f"Processing Image {num_img}")
            images.update()
            
            if folder == ref_imgs and num_img == ref_img :
                continue
            

            img_2 = read_image(
                 f'archive/{str(folder).zfill(3)}/S6{str(folder).zfill(3)}S{str(num_img).zfill(2)}.jpg')

            imgs = [[img_1], [img_2]]
            templates = [[], []]
            masks = [[], []]
            results = []
            

            for i in range(len(imgs)):
                for j in range(len(imgs[i])):
                    img = imgs[i][j]

                    _, snake, circles = localization(img, N=400)

                    pupil_circle = circles
                    iris_circle = np.flip(np.array(snake).astype(int), 1)

                    if circles[2] is None:
                        error += 1
                        break

                    # Image Preprocessing (Normalization)
                    iris_norm = normalization(img, pupil_circle, iris_circle)

                    # Feature Extraction
                    romv_img, noise_img = lash_removal_daugman(
                        iris_norm, thresh=50)
                    template, mask_noise = encode_iris(
                        romv_img, noise_img, minw_length=18, mult=1, sigma_f=0.5)

                    templates[i].append(template)
                    masks[i].append(mask_noise)

                    # Matching
                    if len(templates[1]) > 0:
                        hd_raw = HammingDistance(
                            templates[i-1][j], masks[i-1][j], templates[i][j], masks[i][j])
                        results.append(hd_raw)

            if len(results) == 2:
                data1.append(results)
                if results == []:
                    continue
                if results[0] <= 0.48 :
                    continue
                
                else:
                    error += 1
                    
            else:
                error += 1
                print(f'Folder {folder} Image {num_img}: {results}')
        
        images.reset()

        
        return error/(test_fol * test_img - 1)*100

In [ ]:
def acc_calc_demo3(ref_imgs=0, ref_img=0, test_fol=60, test_img=20):  
    col = ['FolderName'] + ['folder' + str(i) for i in range(test_fol) ]
    rows = [['Image' + str(i)] for i in range(test_img)]
    

    img_1 = read_image(
        f'Iris-Dataset/CASIA-IrisV2/device1/{str(ref_imgs).zfill(4)}/0{str(ref_imgs).zfill(3)}_{str(ref_img).zfill(3)}.bmp')
    


    error = 0
    
    folders = tqdm(range(test_fol), unit='folder', position=0, dynamic_ncols=True,)
    images = tqdm(range(test_img), unit='image', position=1, leave=False, dynamic_ncols=True)
    

    for folder in range(test_fol):
        folders.set_description(f"Processing Folder {folder}")
        folders.update()
        
        for num_img in range(test_img):
            images.set_description(f"Processing Image {num_img}")
            images.update()
            
            if folder == ref_imgs and num_img == ref_img :
                continue
            

            img_2 = read_image(
                 f'Iris-Dataset/CASIA-IrisV2/device1/{str(folder).zfill(4)}/0{str(folder).zfill(3)}_{str(num_img).zfill(3)}.bmp')

            imgs = [[img_1], [img_2]]
            templates = [[], []]
            masks = [[], []]
            results = []
            

            for i in range(len(imgs)):
                for j in range(len(imgs[i])):
                    img = imgs[i][j]

                    _, snake, circles = localization(img, N=400)

                    pupil_circle = circles
                    iris_circle = np.flip(np.array(snake).astype(int), 1)

                    if circles[2] is None:
                        error += 1
                        break

                    # Image Preprocessing (Normalization)
                    iris_norm = normalization(img, pupil_circle, iris_circle)

                    # Feature Extraction
                    romv_img, noise_img = lash_removal_daugman(
                        iris_norm, thresh=50)
                    template, mask_noise = encode_iris(
                        romv_img, noise_img, minw_length=18, mult=1, sigma_f=0.5)

                    templates[i].append(template)
                    masks[i].append(mask_noise)

                    # Matching
                    if len(templates[1]) > 0:
                        hd_raw = HammingDistance(
                            templates[i-1][j], masks[i-1][j], templates[i][j], masks[i][j])
                        results.append(hd_raw)

            if len(results) == 1:
                for value in results:
                    rows[num_img].append(value)
                if results == []:
                    continue
                if results[0] <= 0.48 :
                    continue
                
                else:
                    error += 1
                    
            else:
                error += 1
                print(f'Folder {folder} Image {num_img}: {results}')
                rows[num_img].append(np.nan)
        
        images.reset()
        
    write_iris_data_to_csv(col, rows,'output3.csv', test_fol, test_img)
    
        
           
    return error/(test_fol * test_img-1)*100


In [ ]:
def acc_calc_demo35(ref_imgs=0, ref_img=0, test_fol=60, test_img=20):  
    col = ['FolderName'] + ['folder' + str(i) for i in range(test_fol) ]
    rows = [['Image' + str(i)] for i in range(test_img)]
    
    error = 0
    
    folders = tqdm(range(test_fol), unit='folder', position=0, dynamic_ncols=True,)
    images = tqdm(range(test_img), unit='image', position=1, leave=False, dynamic_ncols=True)
    for ref_imgs in range(ref_imgs):
        for ref_img in range(ref_img):
            img_1 = read_image(
            f'Iris-Dataset/CASIA-IrisV2/device1/{str(ref_imgs).zfill(4)}/0{str(ref_imgs).zfill(3)}_{str(ref_img).zfill(3)}.bmp')

            for folder in range(test_fol):
                folders.set_description(f"Processing Folder {folder}")
                folders.update()
                
                for num_img in range(test_img):
                    images.set_description(f"Processing Image {num_img}")
                    images.update()  

                    
                    
                    if folder == ref_imgs and num_img == ref_img :
                        continue
                    

                    img_2 = read_image(
                        f'Iris-Dataset/CASIA-IrisV2/device1/{str(folder).zfill(4)}/0{str(folder).zfill(3)}_{str(num_img).zfill(3)}.bmp')

                    imgs = [[img_1], [img_2]]
                    templates = [[], []]
                    masks = [[], []]
                    results = []
                    

                    for i in range(len(imgs)):
                        for j in range(len(imgs[i])):
                            img = imgs[i][j]

                            _, snake, circles = localization(img, N=400)

                            pupil_circle = circles
                            iris_circle = np.flip(np.array(snake).astype(int), 1)

                            if circles[2] is None:
                                error += 1
                                break

                            # Image Preprocessing (Normalization)
                            iris_norm = normalization(img, pupil_circle, iris_circle)

                            # Feature Extraction
                            romv_img, noise_img = lash_removal_daugman(
                                iris_norm, thresh=50)
                            template, mask_noise = encode_iris(
                                romv_img, noise_img, minw_length=18, mult=1, sigma_f=0.5)

                            templates[i].append(template)
                            masks[i].append(mask_noise)

                            # Matching
                            if len(templates[1]) > 0:
                                hd_raw = HammingDistance(
                                    templates[i-1][j], masks[i-1][j], templates[i][j], masks[i][j])
                                results.append(hd_raw)

                    if len(results) == 1:
                        for value in results:
                            rows[num_img].append(value)
                        if results == []:
                            continue
                        if results[0] <= 0.48 :
                            continue
                        
                        else:
                            error += 1
                            
                    else:
                        error += 1
                        print(f'Folder {folder} Image {num_img}: {results}')
                        rows[num_img].append(np.nan)
                
                images.reset()
                
            write_iris_data_to_csv(col, rows,'output2.csv', test_fol, test_img)
        
        
           
    return error/(test_fol * test_img-1)*100

In [ ]:
def acc_calc_demo36(ref_imgs=0, ref_img=0, test_fol=60, test_img=20):  
    col = ['FolderName'] + ['folder' + str(i) for i in range(test_fol) ]
    rows = [['Image' + str(i)] for i in range(test_img)]
    classification_results = [] 
    

    img_1 = read_image(
        f'Iris-Dataset/CASIA-IrisV2/device1/{str(ref_imgs).zfill(4)}/0{str(ref_imgs).zfill(3)}_{str(ref_img).zfill(3)}.bmp')
    


    error = 0
    
    folders = tqdm(range(test_fol), unit='folder', position=0, dynamic_ncols=True,)
    images = tqdm(range(test_img), unit='image', position=1, leave=False, dynamic_ncols=True)
    for ref_imgs in range(test_fol):
        for ref_img in range(test_img):
            for folder in range(test_fol):
                for num_img in range(test_img):
                    img_1 = read_image(
                    f'Iris-Dataset/CASIA-IrisV2/device1/{str(ref_imgs).zfill(4)}/0{str(ref_imgs).zfill(3)}_{str(ref_img).zfill(3)}.bmp')

                    for folder in range(test_fol):
                        folders.set_description(f"Processing Folder {folder}")
                        folders.update()
                        
                        for num_img in range(test_img):
                            images.set_description(f"Processing Image {num_img}")
                            images.update()
                            
                            
                            

                            img_2 = read_image(
                                f'Iris-Dataset/CASIA-IrisV2/device1/{str(folder).zfill(4)}/0{str(folder).zfill(3)}_{str(num_img).zfill(3)}.bmp')

                            imgs = [[img_1], [img_2]]
                            templates = [[], []]
                            masks = [[], []]
                            results = []
                            

                            for i in range(len(imgs)):
                                for j in range(len(imgs[i])):
                                    img = imgs[i][j]

                                    _, snake, circles = localization(img, N=400)

                                    pupil_circle = circles
                                    iris_circle = np.flip(np.array(snake).astype(int), 1)

                                    if circles[2] is None:
                                        error += 1
                                        break

                                    # Image Preprocessing (Normalization)
                                    iris_norm = normalization(img, pupil_circle, iris_circle)

                                    # Feature Extraction
                                    romv_img, noise_img = lash_removal_daugman(
                                        iris_norm, thresh=50)
                                    template, mask_noise = encode_iris(
                                        romv_img, noise_img, minw_length=18, mult=1, sigma_f=0.5)

                                    templates[i].append(template)
                                    masks[i].append(mask_noise)

                                    # Matching
                                    if len(templates[1]) > 0:
                                        hd_raw = HammingDistance(
                                            templates[i-1][j], masks[i-1][j], templates[i][j], masks[i][j])
                                        results.append(hd_raw)

                            if len(results) == 1:
                                for value in results:
                                    rows[num_img].append(value)
                                if results == []:
                                    continue
                                if results[0] <= 0.48 :
                                    continue
                                
                                else:
                                    error += 1
                                    
                            else:
                                error += 1
                                print(f'Folder {folder} Image {num_img}: {results}')
                                rows[num_img].append(np.nan)
                        
                        images.reset()
                        
                            
                for num_img in range(test_img):
                    classification_results.append([])
                    for value in results:
                        if value <= 0.48:  
                            classification = calc_classification_metrics(1, 1)  
                        else:
                            classification = calc_classification_metrics(1, 0)  
                        classification_results[num_img].append(classification)

                
                for i in range(test_img):
                    rows[i].extend(classification_results[i])    
                    write_iris_data_to_csv(col, rows,'output2.csv', test_fol, test_img)
            
        
           
    return error/(test_fol * test_img-1)*100

In [ ]:
def acc_calc_demo37(ref_imgs=0, ref_img=0, test_fol=60, test_img=20):  
    col = ['FolderName'] + ['folder' + str(i) for i in range(test_fol) ]
    rows = [['Image' + str(i)] for i in range(test_img)]
    tp_count = 0
    tn_count = 0
    fp_count = 0
    fn_count = 0
    

    img_1 = read_image(
        f'Iris-Dataset/CASIA-IrisV2/device1/{str(ref_imgs).zfill(4)}/0{str(ref_imgs).zfill(3)}_{str(ref_img).zfill(3)}.bmp')
    


    error = 0
    
    folders = tqdm(range(test_fol), unit='folder', position=0, dynamic_ncols=True,)
    images = tqdm(range(test_img), unit='image', position=1, leave=False, dynamic_ncols=True)
    

    for folder in range(test_fol):
        folders.set_description(f"Processing Folder {folder}")
        folders.update()
        
        for num_img in range(test_img):
            images.set_description(f"Processing Image {num_img}")
            images.update()
            for ref_imgs in range(test_fol):
              for ref_img in range(test_img):
                  if folder != ref_imgs or num_img != ref_img:
                      img_2 = read_image(
                          f'Iris-Dataset/CASIA-IrisV2/device1/{str(ref_imgs).zfill(4)}/0{str(ref_imgs).zfill(3)}_{str(ref_img).zfill(3)}.bmp')
            if folder == ref_imgs and num_img == ref_img :
                continue
            

            img_2 = read_image(
                 f'Iris-Dataset/CASIA-IrisV2/device1/{str(folder).zfill(4)}/0{str(folder).zfill(3)}_{str(num_img).zfill(3)}.bmp')

            imgs = [[img_1], [img_2]]
            templates = [[], []]
            masks = [[], []]
            results = []
            

            for i in range(len(imgs)):
                for j in range(len(imgs[i])):
                    img = imgs[i][j]

                    _, snake, circles = localization(img, N=400)

                    pupil_circle = circles
                    iris_circle = np.flip(np.array(snake).astype(int), 1)

                    if circles[2] is None:
                        error += 1
                        break

                    # Image Preprocessing (Normalization)
                    iris_norm = normalization(img, pupil_circle, iris_circle)

                    # Feature Extraction
                    romv_img, noise_img = lash_removal_daugman(
                        iris_norm, thresh=50)
                    template, mask_noise = encode_iris(
                        romv_img, noise_img, minw_length=18, mult=1, sigma_f=0.5)

                    templates[i].append(template)
                    masks[i].append(mask_noise)

                    # Matching
                    if len(templates[1]) > 0:
                        hd_raw = HammingDistance(
                            templates[i-1][j], masks[i-1][j], templates[i][j], masks[i][j])
                        results.append(hd_raw)

            if len(results) == 1:
                for value in results:
                    rows[num_img].append(value)
                if results == []:
                    continue
                if results[0] <= 0.48 :
                    continue
                
                else:
                    error += 1
                    
            else:
                error += 1
                print(f'Folder {folder} Image {num_img}: {results}')
                rows[num_img].append(np.nan)
        for num_img in range(test_img):
            for value in results:
                if value <= 0.48:
                    classification = calc_classification_metrics(1, 1)
                    if classification == "TP":
                        tp_count += 1
                    elif classification == "TN":
                        tn_count += 1  # Assuming all non-reference are negative
                else:
                    classification = calc_classification_metrics(1, 0)
                    if classification == "FN":
                        fn_count += 1
                    elif classification == "FP":
                        fp_count += 1
            print("TP:", tp_count)
            print("TN:", tn_count)
            print("FP:", fp_count)
            print("FN:", fn_count)
            images.reset()
        
    write_iris_data_to_csv(col, rows,'output2.csv', test_fol, test_img)
    
        
           
    return error/(test_fol * test_img-1)*100

In [ ]:
def acc_calc_demo38(ref_imgs=0, ref_img=0, test_fol=60, test_img=20):  
    col = ['FolderName'] + ['folder' + str(i) for i in range(test_fol) ]
    rows = [['Image' + str(i)] for i in range(test_img)]
    tp_count = 0
    tn_count = 0
    fp_count = 0
    fn_count = 0
    

    img_1 = read_image(
        f'Iris-Dataset/CASIA-IrisV2/device1/{str(ref_imgs).zfill(4)}/0{str(ref_imgs).zfill(3)}_{str(ref_img).zfill(3)}.bmp')
    


    error = 0
    
    folders = tqdm(range(test_fol), unit='folder', position=0, dynamic_ncols=True,)
    images = tqdm(range(test_img), unit='image', position=1, leave=False, dynamic_ncols=True)
    thresholds = np.arange(0.0, 1.01, 0.01)
    false_positive_rates = []
    false_negative_rates = []

    for folder in range(test_fol):
        folders.set_description(f"Processing Folder {folder}")
        folders.update()
        
        for num_img in range(test_img):
            images.set_description(f"Processing Image {num_img}")
            images.update()
            for ref_imgs in range(test_fol):
              for ref_img in range(test_img):
                  if folder != ref_imgs or num_img != ref_img:
                      img_2 = read_image(
                          f'Iris-Dataset/CASIA-IrisV2/device1/{str(ref_imgs).zfill(4)}/0{str(ref_imgs).zfill(3)}_{str(ref_img).zfill(3)}.bmp')
            if folder == ref_imgs and num_img == ref_img :
                continue
            

            img_2 = read_image(
                 f'Iris-Dataset/CASIA-IrisV2/device1/{str(folder).zfill(4)}/0{str(folder).zfill(3)}_{str(num_img).zfill(3)}.bmp')

            imgs = [[img_1], [img_2]]
            templates = [[], []]
            masks = [[], []]
            results = []
            

            for i in range(len(imgs)):
                for j in range(len(imgs[i])):
                    img = imgs[i][j]

                    _, snake, circles = localization(img, N=400)

                    pupil_circle = circles
                    iris_circle = np.flip(np.array(snake).astype(int), 1)

                    if circles[2] is None:
                        error += 1
                        break

                    # Image Preprocessing (Normalization)
                    iris_norm = normalization(img, pupil_circle, iris_circle)

                    # Feature Extraction
                    romv_img, noise_img = lash_removal_daugman(
                        iris_norm, thresh=50)
                    template, mask_noise = encode_iris(
                        romv_img, noise_img, minw_length=18, mult=1, sigma_f=0.5)

                    templates[i].append(template)
                    masks[i].append(mask_noise)

                    # Matching
                    if len(templates[1]) > 0:
                        hd_raw = HammingDistance(
                            templates[i-1][j], masks[i-1][j], templates[i][j], masks[i][j])
                        results.append(hd_raw)

            if len(results) == 1:
                for value in results:
                    rows[num_img].append(value)
                if results == []:
                    continue
                if results[0] <= 0.48 :
                    continue
                
                else:
                    error += 1
                    
            else:
                error += 1
                print(f'Folder {folder} Image {num_img}: {results}')
                rows[num_img].append(np.nan)
        for num_img in range(test_img):
            for value in results:
                if value <= 0.48:
                    classification = calc_classification_metrics(1, 1)
                    if classification == "TP":
                        tp_count += 1
                    elif classification == "TN":
                        tn_count += 1  # Assuming all non-reference are negative
                else:
                    classification = calc_classification_metrics(1, 0)
                    if classification == "FN":
                        fn_count += 1
                    elif classification == "FP":
                        fp_count += 1
        print("TP:", tp_count)
        print("TN:", tn_count)
        print("FP:", fp_count)
        print("FN:", fn_count)
                        

                        
        plt.figure()
        plt.plot(fp_count, fn_count, label='ROC Curve')
        plt.xlabel('False Positive ')
        plt.ylabel('False Negative ')
        plt.title('ROC Curve for Iris Recognition')
        plt.grid(True)
        plt.show()
        images.reset()
        
    write_iris_data_to_csv(col, rows,'output2.csv', test_fol, test_img)
    
        
           
    return error/(test_fol * test_img-1)*100

In [ ]:

def acc_calc_demo39(ref_imgs=0, ref_img=0, test_fol=5, test_img=2): 
    
    col = ['FolderName'] + ['folder' + str(i) for i in range(test_fol) ]
    rows = [['Image' + str(i)] for i in range(test_img)]
    tp_count = 0
    tn_count = 0
    fp_count = 0
    fn_count = 0
    fn_list = []
    fp_list = []
    

    img_1 = read_image(
        f'Iris-Dataset/CASIA-IrisV2/device1/{str(ref_imgs).zfill(4)}/0{str(ref_imgs).zfill(3)}_{str(ref_img).zfill(3)}.bmp')
    


    error = 0
    
    folders = tqdm(range(test_fol), unit='folder', position=0, dynamic_ncols=True,)
    images = tqdm(range(test_img), unit='image', position=1, leave=False, dynamic_ncols=True)
    
    

    for folder in range(test_fol):
        folders.set_description(f"Processing Folder {folder}")
        folders.update()
        
        for num_img in range(test_img):
            images.set_description(f"Processing Image {num_img}")
            images.update()
            for ref_imgs in range(test_fol):
              for ref_img in range(test_img):
                  if folder != ref_imgs or num_img != ref_img:
                      img_2 = read_image(
                          f'Iris-Dataset/CASIA-IrisV2/device1/{str(ref_imgs).zfill(4)}/0{str(ref_imgs).zfill(3)}_{str(ref_img).zfill(3)}.bmp')
            if folder == ref_imgs and num_img == ref_img :
                continue
            

            img_2 = read_image(
                 f'Iris-Dataset/CASIA-IrisV2/device1/{str(folder).zfill(4)}/0{str(folder).zfill(3)}_{str(num_img).zfill(3)}.bmp')

            imgs = [[img_1], [img_2]]
            templates = [[], []]
            
            masks = [[], []]
            results = []
            

            for i in range(len(imgs)):
                for j in range(len(imgs[i])):
                    img = imgs[i][j]

                    _, snake, circles = localization(img, N=400)

                    pupil_circle = circles
                    iris_circle = np.flip(np.array(snake).astype(int), 1)

                    if circles[2] is None:
                        error += 1
                        break

                    # Image Preprocessing (Normalization)
                    iris_norm = normalization(img, pupil_circle, iris_circle)

                    # Feature Extraction
                    romv_img, noise_img = lash_removal_daugman(
                        iris_norm, thresh=50)
                    template, mask_noise = encode_iris(
                        romv_img, noise_img, minw_length=18, mult=1, sigma_f=0.5)

                    templates[i].append(template)
                    masks[i].append(mask_noise)

                    # Matching
                    if len(templates[1]) > 0:
                        hd_raw = HammingDistance(
                            templates[i-1][j], masks[i-1][j], templates[i][j], masks[i][j])
                        results.append(hd_raw)

            if len(results) == 1:
                for value in results:
                    rows[num_img].append(value)
                if results == []:
                    continue
                if results[0] <= 0 :
                    continue
                
                else:
                    error += 1
                    
            else:
                error += 1
                print(f'Folder {folder} Image {num_img}: {results}')
                rows[num_img].append(np.nan)
        for num_img in range(test_img):
            for value in results:
                if value <= 0.4725:
                    classification = calc_classification_metrics(1, 1)
                    if classification == "TP":
                        tp_count += 1
                    elif classification == "TN":
                        tn_count += 1  
                else:
                    classification = calc_classification_metrics(1, 0)
                    if classification == "FN":
                        fn_count += 1
                        fn_list.append(fn_count)
                    elif classification == "FP":
                        fp_count += 1
                        fp_list.append(fp_count)
            print("TP:", tp_count)
            print("TN:", tn_count)
            print("FP:", fp_count)
            print("FN:", fn_count)
            images.reset()
        
    write_iris_data_to_csv(col, rows,'output3.csv', test_fol, test_img)
    plt.plot(range(len(fn_list)), fn_list, label='False Negatives')
    plt.plot(range(len(fp_list)), fp_list, label='False Positives')
    plt.xlabel('Iterations')
    plt.ylabel('Count')
    plt.title('FN vs FP')
    plt.legend()
    plt.show()
        
           
    return error/(test_fol * test_img-1)*100
    

In [ ]:
def acc_calc_demo5(ref_imgs=0, ref_img=0, test_fol=100, test_img=10):  
    col = ['FolderName'] + ['folder' + str(i) + 'L' if j % 2 == 0 else 'folder' + str(i) + 'R' for i in range(test_fol) for j in range(2) ]
    rows = [['Image' + str(i)] for i in range(test_img)]
    
    img_1_L = read_image(
         f'CASIA-Iris-Interval/{str(ref_imgs).zfill(3)}/L/S1{str(ref_imgs).zfill(3)}L{str(ref_img).zfill(2)}.jpg')
        
    img_1_R = read_image(
         f'CASIA-Iris-Interval/{str(ref_imgs).zfill(3)}/R/S1{str(ref_imgs).zfill(3)}R{str(ref_img).zfill(2)}.jpg')
    
        

    error = 0
    
    folders = tqdm(range(test_fol), unit='folder', position=0, dynamic_ncols=True,)
    images = tqdm(range(test_img), unit='image', position=1, leave=False, dynamic_ncols=True)
    
    for folder in range(test_fol):
        folders.set_description(f"Processing Folder {folder}")
        folders.update()
        
        for num_img in range(test_img):
            images.set_description(f"Processing Image {num_img}")
            images.update()
            
            if folder == ref_imgs and num_img == ref_img:
                continue
            img_2_L = read_image(
            f'CASIA-Iris-Interval/{str(folder).zfill(3)}/L/S1{str(folder).zfill(3)}L{str(num_img).zfill(2)}.jpg')
            img_2_R = read_image(
            f'CASIA-Iris-Interval/{str(folder).zfill(3)}/R/S1{str(folder).zfill(3)}R{str(num_img).zfill(2)}.jpg')

            imgs = [[img_1_L, img_1_R], [img_2_L, img_2_R]]
            templates = [[], []]
            masks = [[], []]
            results = []

            for i in range(len(imgs)):
                for j in range(len(imgs[i])):
                    img = imgs[i][j]

                    _, snake, circles = localization(img, N=400)

                    pupil_circle = circles
                    iris_circle = np.flip(np.array(snake).astype(int), 1)

                    if circles[2] is None:
                        error += 1
                        break

                    # Image Preprocessing (Normalization)
                    iris_norm = normalization(img, pupil_circle, iris_circle)

                    # Feature Extraction
                    romv_img, noise_img = lash_removal_daugman(
                        iris_norm, thresh=50)
                    template, mask_noise = encode_iris(
                        romv_img, noise_img, minw_length=18, mult=1, sigma_f=0.5)
                    


                    templates[i].append(template)
                    masks[i].append(mask_noise)

                    # Matching
                    if len(templates[1]) > 0:
                        hd_raw = HammingDistance(
                            templates[i-1][j], masks[i-1][j], templates[i][j], masks[i][j])
                        results.append(hd_raw)

            if len(results) == 2:
                for value in results:
                    rows[num_img].append(value)
                
                if results[0] <= 0.48 and results[1] <= 0.48:
                    if folder == ref_imgs:
                        continue
                    else:
                        error += 1
                elif (results[0] <= 0.49 and results[1] <= 0.45) or (results[1] <= 0.49 and results[0] <= 0.45):
                    if folder == ref_imgs:
                        continue
                    else:
                        error += 1
                else:
                    continue
            else:
                error += 1
                print(f'Folder {folder} Image {num_img}: {results}')
                for j in range(1):
                    rows[num_img].append(np.nan)
    
        images.reset()

    write_iris_data_to_csv(col, rows,'output3.csv', test_fol, test_img)
        
           
    return error/(test_fol * test_img-1)*100


In [ ]:
def acc_calc_demo01(ref_imgs=0, ref_img=0, test_fol=100, test_img=10):  
    col = ['FolderName'] + ['folder' + str(i) for i in range(test_fol) ]
    rows = [['Image' + str(i)] for i in range(test_img)]
    
    img_1 = read_image(
        f'archive/{str(ref_imgs).zfill(3)}/S6{str(ref_imgs).zfill(3)}S{str(ref_img).zfill(2)}.jpg')
    


    error = 0
    
    folders = tqdm(range(test_fol), unit='folder', position=0, dynamic_ncols=True,)
    images = tqdm(range(test_img), unit='image', position=1, leave=False, dynamic_ncols=True)
    
    for folder in range(test_fol):
        folders.set_description(f"Processing Folder {folder}")
        folders.update()
        
        for num_img in range(test_img):
            images.set_description(f"Processing Image {num_img}")
            images.update()
            
            if folder == ref_imgs and num_img == ref_img :
                continue
            

            img_2 = read_image(
                 f'archive/{str(folder).zfill(3)}/S6{str(folder).zfill(3)}S{str(num_img).zfill(2)}.jpg')

            imgs = [[img_1], [img_2]]
            templates = [[], []]
            masks = [[], []]
            results = []
            

            for i in range(len(imgs)):
                for j in range(len(imgs[i])):
                    img = imgs[i][j]

                    _, snake, circles = localization(img, N=400)

                    pupil_circle = circles
                    iris_circle = np.flip(np.array(snake).astype(int), 1)

                    if circles[2] is None:
                        error += 1
                        break

                    # Image Preprocessing (Normalization)
                    iris_norm = normalization(img, pupil_circle, iris_circle)

                    # Feature Extraction
                    romv_img, noise_img = lash_removal_daugman(
                        iris_norm, thresh=50)
                    template, mask_noise = encode_iris(
                        romv_img, noise_img, minw_length=18, mult=1, sigma_f=0.5)

                    templates[i].append(template)
                    masks[i].append(mask_noise)

                    # Matching
                    if len(templates[1]) > 0:
                        hd_raw = HammingDistance(
                            templates[i-1][j], masks[i-1][j], templates[i][j], masks[i][j])
                        results.append(hd_raw)

            if len(results) == 1:
                for value in results:
                    rows[num_img].append(value)
                if results == []:
                    continue
                if results[0] <= 0.48 :
                    continue
                
                else:
                    error += 1
                    
            else:
                error += 1
                print(f'Folder {folder} Image {num_img}: {results}')
                rows[num_img].append(np.nan)
        
        images.reset()
    write_iris_data_to_csv(col, rows,'output3.csv', test_fol, test_img)
    
        
           
    return error/(test_fol * test_img-1)*100


In [ ]:
def acc_calc(ref_imgs=0, ref_img=0, test_fol=100, test_img=10):    
    img_1_L = read_image(
        f'Iris-Dataset/CASIA-Iris-Thousand/{str(ref_imgs).zfill(3)}/L/S5{str(ref_imgs).zfill(3)}L{str(ref_img).zfill(2)}.jpg')

    img_1_R = read_image(
        f'Iris-Dataset/CASIA-Iris-Thousand/{str(ref_imgs).zfill(3)}/R/S5{str(ref_imgs).zfill(3)}R{str(ref_img).zfill(2)}.jpg')

    error = 0
    
    folders = tqdm(range(test_fol), unit='folder', position=0, dynamic_ncols=True,)
    images = tqdm(range(test_img), unit='image', position=1, leave=False, dynamic_ncols=True)
    
    for folder in range(test_fol):
        folders.set_description(f"Processing Folder {folder}")
        folders.update()
        
        for num_img in range(test_img):
            images.set_description(f"Processing Image {num_img}")
            images.update()
            
            if folder == ref_imgs and num_img == ref_img:
                continue
            img_2_L = read_image(
                f'Iris-Dataset/CASIA-Iris-Thousand/{str(folder).zfill(3)}/L/S5{str(folder).zfill(3)}L{str(num_img).zfill(2)}.jpg')

            img_2_R = read_image(
                f'Iris-Dataset/CASIA-Iris-Thousand/{str(folder).zfill(3)}/R/S5{str(folder).zfill(3)}R{str(num_img).zfill(2)}.jpg')

            imgs = [[img_1_L, img_1_R], [img_2_L, img_2_R]]
            templates = [[], []]
            masks = [[], []]
            results = []

            for i in range(len(imgs)):
                for j in range(len(imgs[i])):
                    img = imgs[i][j]

                    _, snake, circles = localization(img, N=400)

                    pupil_circle = circles
                    iris_circle = np.flip(np.array(snake).astype(int), 1)

                    if circles[2] is None:
                        error += 1
                        break

                    # Image Preprocessing (Normalization)
                    iris_norm = normalization(img, pupil_circle, iris_circle)

                    # Feature Extraction
                    romv_img, noise_img = lash_removal_daugman(
                        iris_norm, thresh=50)
                    template, mask_noise = encode_iris(
                        romv_img, noise_img, minw_length=18, mult=1, sigma_f=0.5)

                    templates[i].append(template)
                    masks[i].append(mask_noise)

                    # Matching
                    if len(templates[1]) > 0:
                        hd_raw = HammingDistance(
                            templates[i-1][j], masks[i-1][j], templates[i][j], masks[i][j])
                        results.append(hd_raw)

            if len(results) == 2:
                if results == []:
                    continue
                if results[0] <= 0.48 and results[1] <= 0.48:
                    error += 1
                elif (results[0] <= 0.49 and results[1] <= 0.45) or (results[1] <= 0.49 and results[0] <= 0.45):
                    error += 1
                else:
                    continue
            else:
                error += 1
                print(f'Folder {folder} Image {num_img}: {results}')
        
        images.reset()
        
    return error/999*100

In [ ]:
def get_param(file = 'output3.csv', threshold = 0.48):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    df = pd.read_csv(file)
    for i in range(0, df.shape[1], 10):
        test = np.array(df.iloc[i:i + 10, i: i+10])
        temp1 = test <= threshold
        TP = TP + (temp1.sum() - 10)
        temp2 = test > threshold
        FN = FN + (temp2.sum())

    df = np.array(df)
    df_temp1 = df <= threshold
    FP = df_temp1.sum() - df.shape[1] - TP
    df_temp2 = df > threshold
    TN = df_temp2.sum() - FN

    return np.array([[TP, FP], [FN, TN]])


In [ ]:
from scipy import interpolate
from scipy.optimize import fsolve
test = np.linspace(0.25, 0.48, 40)
FAR = []
FRR = []
for i in test:
    
    temp = get_param(file = 'output3.csv', threshold=i)
    FAR.append( temp[0, 1]/sum(temp[0]))
    FRR.append( temp[1, 0]/sum(temp[1]))


far_spline = interpolate.splrep(test, FAR)
frr_spline = interpolate.splrep(test, FRR)


def difference(x):
    far_value = interpolate.splev(x, far_spline)
    frr_value = interpolate.splev(x, frr_spline)
    return far_value - frr_value

initial_guess = 0
threshold_intersection = fsolve(difference, initial_guess)
print(f'threshold:{threshold_intersection[0]:.3f}, intersection:{interpolate.splev(threshold_intersection[0], far_spline):.3f}')

FAR = np.array(FAR)
FRR = np.array(FRR)

plt.figure(figsize=(8, 6), dpi=80)
plt.plot(test, FAR, label ='FAR')
plt.plot(test, FRR, label = 'FRR')
plt.title("FPR and FNR vs Threshold")
plt.xlabel("Threshold")
plt.ylabel("Rate")
plt.legend()
plt.show()

## <span style="color:orange">*Main Program*</span>

#plot_prepro(img_num1=0, img_num2=40, plot_cols=2)
#plot_matching(img_num=0, compare_num=20)
#ham_dist = acc_plot(img_num=2, ref_img='CASIA-IrisV2/device1/0000/0000_000.bmp')
accuracy = 100 - acc_calc_demo37(ref_imgs=0, ref_img=0, test_fol=60, test_img=20)

In [ ]:
# test_img = '/content/file_folder_drive'

In [ ]:
# path = '/content/gdrive/MyDrive/Iris Recognition/Share Iris/Iris-Dataset/CASIA-Iris-Thousand'

In [ ]:
#plot_prepro(img_num1=0, img_num2=40, plot_cols=2)
#plot_matching(img_num=0, compare_num=20)
#ham_dist = acc_plot(img_num=2, ref_img='CASIA-IrisV2/device1/0000/0000_000.bmp')
accuracy = 100 - acc_calc_demo35(ref_imgs=0, ref_img=0, test_fol=2, test_img=2)  

#plot_prepro(img_num1=0, img_num2=40, plot_cols=2)
#plot_matching(img_num=0, compare_num=20)
#ham_dist = acc_plot(img_num=2, ref_img='CASIA-IrisV2/device1/0000/0000_000.bmp')
accuracy = 100 - acc_calc(ref_imgs=0, ref_img=0, test_fol=200, test_img=5)

In [ ]:
print(f'Accuracy: {round(accuracy)}%')

In [ ]:
# ham_dist.to_csv('Output/0000_000.csv')

In [ ]:
ham_dist = pd.read_csv('output3.csv', index_col=0)

In [ ]:
ham_dist.style.highlight_null().highlight_min(color='green').highlight_max(color='blue').applymap(lambda x: 'background-color: white' if x == 0 else '').format("{:.3f}")

In [ ]:
ham_dist.style.applymap(lambda x: 'background-color: red' if x <= 0.48 else '').format("{:.3f}").highlight_null()

In [ ]:
mat_diff = ham_dist.iloc[:, 1:].copy()
mat_same = ham_dist.iloc[:, 0].copy()

mat_diff = np.where(mat_diff > 0.46, 1, 0)
mat_same = np.where(mat_same <= 0.46, 1, 0)

In [ ]:
acc_diff = np.sum(mat_diff) / (mat_diff.shape[0] * mat_diff.shape[1])
acc_same = np.sum(mat_same) / mat_same.shape[0]

print(f'Accuracy for different image: {acc_diff*100:.2f}%')
print(f'Accuracy for same image: {acc_same*100:.2f}%')